# Table of Contents
 <p>

In [17]:
%load_ext autoreload
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
%autoreload 2

import numpy as np
import numpy.ma as ma
import dask.array as da
import matplotlib.pyplot as plt
import datetime as dt
import sys,os,glob
from mpl_toolkits.basemap import Basemap
from matplotlib.colors import LogNorm

## Add own library to path
workdir = os.getcwd()
thismodule = sys.modules[__name__]
moduledir = os.path.join(os.path.dirname(workdir),'functions')
sys.path.insert(0,moduledir)
print("Own modules available:", [os.path.splitext(os.path.basename(x))[0]
                                 for x in glob.glob(os.path.join(moduledir,'*.py'))])
## Load own libraries
from environmentAndDirectories import *
from importingData import *
from scalingApproximations import *
from slicingAndSubsetting import *
from thermoConstants import L_v,R_v
from plotMaps import *
from plot1DInvLog import *
from statisticalDistributions import *

Own modules available: ['CAMsettings', 'daskOptions', 'environmentAndDirectories', 'importingData', 'physicalConstants', 'plot1DInvLog', 'plotMaps', 'scalingApproximations', 'slicingAndSubsetting', 'statisticalDistributions', 'thermoConstants', 'thermoFunctions']


In [83]:
n_pts = int(1e4)
ts = np.random.normal(300,20,n_pts)
ps = 1e5*np.ones(n_pts)
omega = np.random.normal(0.1,0.5,n_pts)
qvstar = saturationSpecificHumidity(ts,ps)
pr_sc = omega*qvstar

from scipy.stats import norm
pr = np.random.lognormal(mean=0.1*qvstar.mean(),sigma = sqrt(np.log(qvstar).var()),size=n_pts)

In [84]:
print(ranks[iQ_slice])

[ 99.74881136  99.80047377  99.84151068  99.87410746  99.9         99.92056718
  99.93690427  99.94988128  99.96018928  99.96837722]


In [103]:
Q_IL = getInvLogRanks(n_pts,fill_last_decade=True)
i_Q = indexOfQ(99.9,Q_IL)
ranks, percentiles, breaks = computePercentilesAndBinsFromRanks(pr.flatten(),Q_IL)
iQ_slice = slice(i_Q-5,i_Q+5)
eff, pr_sc_vQ = computeScalingOmegaTAtAllRanks(ranks,percentiles,omega,ts,ps,pr,ranks[iQ_slice])
print(eff)

-71907.5524135


In [97]:
print("Compute variance in percentiles bins: prefix 'var_', suffix '_prQ'")
var_pr_sc_prQ = np.array(list(map(lambda x: varXAtYPercentiles(x,ranks,percentiles,pr_sc,pr),ranks)))
var_ts_prQ = np.array(list(map(lambda x: varXAtYPercentiles(x,ranks,percentiles,ts,pr),ranks)))
var_omega_prQ = np.array(list(map(lambda x: varXAtYPercentiles(x,ranks,percentiles,omega,pr),ranks)))

Compute variance in percentiles bins - quantity to decompose: prefix 'var_', suffix '_prQ'


In [100]:
# print("Compute covariance in percentile bins: prefix 'cov_', suffix '_prQ'")
# cov_ts_omega_prQ = np.array(list(map(lambda x: covAtYPercentiles(x,ranks,percentiles,ts,omega,pr),ranks)))

Compute covariance in percentile bins: prefix 'cov_', suffix '_prQ'


In [93]:
print("Compute all mean variables in percentile bins of pr_ref (pr): suffix '_prQ'")
ts_prQ = np.array(list(map(lambda x: meanXAtYPercentiles(x,ranks,percentiles,ts,pr),ranks)))
ps_prQ = np.array(list(map(lambda x: meanXAtYPercentiles(x,ranks,percentiles,ps,pr),ranks)))
omega_prQ = np.array(list(map(lambda x: meanXAtYPercentiles(x,ranks,percentiles,omega,pr),ranks)))

Compute all mean variables in percentile bins of pr_ref (pr): suffix '_prQ'


In [102]:
print("Compute functions of binned variables: suffix '_vQ'")
pr_sc_vQ = singleLevelScalingFromOmegaT(omega_prQ,ts_prQ,ps_prQ,eff)
qvstar_vQ = saturationSpecificHumidity(ts_prQ,ps_prQ)

Compute functions of binned variables: suffix '_vQ'


In [111]:
print("Compute coefficients in the variance decomposition")
a = (eff*qvstar_vQ)**2
b = (omega_prQ*L_v/(R_v*ts_prQ**2))**2
c_varT = a*b
c_varOmega = a
# print('c_varT =',c_varT)
# print('c_varOmega =',c_varOmega)

Compute coefficients in the variance decomposition


In [110]:
print("Compute normalized terms in the variance decomposition")
term_var_pr_sc = var_pr_sc_prQ/pr_sc_vQ**2
term_varT = c_varT*var_ts_prQ/pr_sc_vQ**2
term_varOmega = c_varOmega*var_omega_prQ/pr_sc_vQ**2

Compute normalized terms in the variance decomposition


In [107]:
print(term_var_pr_sc,term_varT,term_varOmega)

[  5.30655616e-05   6.45205503e-05   4.78777221e-05   5.94554008e-05
   4.94828055e-05   5.72460161e-05   5.65308283e-05   4.58654028e-05
   4.16033317e-05   3.83274123e-05   4.10883620e-05   3.02600671e-05
   3.54454286e-05   4.69515680e-04   4.08472742e-05   7.07658881e-05
   2.64479970e-05   3.10927316e-05  -4.21198487e-04   2.24468230e-05
   6.74046899e-05   6.83214968e-06  -7.64984329e-05   2.28929843e-05
   1.63103653e-05   7.60953657e-06  -4.56859654e-06  -8.15809392e-03
  -4.25158831e-07   2.51652928e-06  -2.87345095e-06              nan
              nan   1.67648626e-07              nan              nan
              nan              nan              nan] [ 2313.96890227  2195.23259744  2720.06226499  1996.24620488  2393.69572787
  2433.59917929  2064.09503783  2398.56142902  2007.54627089  4306.30199614
  2428.58973513  2395.54028489  1396.09785331   114.63243029  4482.19196467
  1570.29142428  2939.95157285  1197.42485002  -168.67601892  1064.59366611
   949.6720246   1766.